In [1]:
import cv2
import numpy as np
import math

In [2]:
class ObjectDetection:
    def __init__(self, weights_path="dnn_model/yolov4.weights", cfg_path="dnn_model/yolov4.cfg"):
        # Print loading messages
        print("Loading Object Detection")
        print("Running opencv dnn with YOLOv4")
        
        # Set the Non-Maximum Suppression threshold, Confidence threshold, and Image size
        self.nmsThreshold = 0.4
        self.confThreshold = 0.5
        self.image_size = 608

        # Load the YOLOv4 model
        net = cv2.dnn.readNet(weights_path, cfg_path)

        # Enable GPU acceleration with CUDA
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        self.model = cv2.dnn_DetectionModel(net)

        # Initialize the list of class names and load them
        self.classes = []
        self.load_class_names()
        
        # Assign random colors for each class
        self.colors = np.random.uniform(0, 255, size=(80, 3))

        # Set input parameters for the model
        self.model.setInputParams(size=(self.image_size, self.image_size), scale=1/255)

    def load_class_names(self, classes_path="dnn_model/classes.txt"):
        # Load class names from a text file
        with open(classes_path, "r") as file_object:
            for class_name in file_object.readlines():
                class_name = class_name.strip()
                self.classes.append(class_name)

        # Assign random colors for each class
        self.colors = np.random.uniform(0, 255, size=(80, 3))
        return self.classes

    def detect(self, frame):
        # Perform object detection on a given frame
        return self.model.detect(frame, nmsThreshold=self.nmsThreshold, confThreshold=self.confThreshold)

In [3]:
# Initialize Object Detection
od = ObjectDetection()

# Open the video file
cap = cv2.VideoCapture("traffic.mp4")

# Initialize count
count = 0
# Initialize list to store center points of detected objects in the previous frame
center_points_prev_frame = []

# Initialize dictionary to store tracking objects
tracking_objects = {}
# Initialize tracking id
track_id = 0

# Loop over the frames of the video
while True:
    # Read the next frame from the video
    ret, frame = cap.read()
    # Increment the frame count
    count += 1
    # If the frame could not be grabbed, then we have reached the end of the video
    if not ret:
        break

    # Initialize list to store center points of detected objects in the current frame
    center_points_cur_frame = []

    # Detect objects on the current frame
    (class_ids, scores, boxes) = od.detect(frame)
    # Loop over the bounding boxes
    for box in boxes:
        # Extract the bounding box coordinates
        (x, y, w, h) = box
        # Compute the center of the bounding box
        cx = int((x + x + w) / 2)
        cy = int((y + y + h) / 2)
        # Add the center point to the list
        center_points_cur_frame.append((cx, cy))

        # Draw the bounding box on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # If this is one of the first two frames, compare the center points of the current and previous frames
    if count <= 2:
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                # Compute the Euclidean distance between the two points
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                # If the distance is less than 20 pixels, add the point to the tracking objects
                if distance < 20:
                    tracking_objects[track_id] = pt
                    track_id += 1
    else:
        # For the rest of the frames, update the tracking objects
        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()

        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            for pt in center_points_cur_frame_copy:
                # Compute the Euclidean distance between the two points
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                # If the distance is less than 20 pixels, update the position of the tracking object
                if distance < 20:
                    tracking_objects[object_id] = pt
                    object_exists = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                    continue

            # If the tracking object does not exist in the current frame, remove it
            if not object_exists:
                tracking_objects.pop(object_id)

        # Add new tracking objects found in the current frame
        for pt in center_points_cur_frame:
            tracking_objects[track_id] = pt
            track_id += 1

    # Draw the tracking objects on the frame
    for object_id, pt in tracking_objects.items():
        cv2.circle(frame, pt, 5, (0, 0, 255), -1)
        cv2.putText(frame, str(object_id), (pt[0], pt[1] - 7), 0, 1, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Update the list of center points for the next iteration
    center_points_prev_frame = center_points_cur_frame.copy()

    # If the 'q' key is pressed, break from the loop
    key = cv2.waitKey(1)
    if key == 27: 
        break

# Release the video file
cap.release()
# Close all OpenCV windows
cv2.destroyAllWindows()

Loading Object Detection
Running opencv dnn with YOLOv4
